In [1]:
import random
import gymnasium as gym
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from helper_functions import reward_print, print_Qtable
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "cpu"
)
device = "cpu"
print(device)
CUDA_LAUNCH_BLOCKING=1

cpu


In [28]:
class Net(nn.Module):

    def __init__(self, obs, action):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(obs, 32)
        self.layer2 = nn.Linear(32, 32)
        self.layer3 = nn.Linear(32, action)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [27]:
# AC2 algorithm 
class AC2:
    def __init__(self, env):
        self.actor = Actor(env)
        self.critic = Critic(self.actor.obs)
    
    # Main training loop
    def train(self, episodes, gamma, greedy=False):
        total_reward = [0] * episodes
        for i in range(episodes):
            step = rewards = 0
            done = False            
            while not done:
                # Actor makes decision 
                action = self.actor.act(self, state)
                # Environment returns state and reward
                next_state, reward, terminated, truncated, _ = self.actor.step(action)
                done = terminated or truncated 
                # Critic evaluates action 
                value = self.critic.evaluate(next_state, reward)
                # Pass that value to the Actor
                self.actor.evaluation(value)
                
                step += 1
                rewards += reward
                
            total_reward[i] = rewards      
        return total_reward

               
    def save(self, filename):
        placeholder = ''
        with open("pickles/" + filename, 'wb') as file:
            pickle.dump(placeholder, file)

In [24]:
# Actor thread
class Actor:
    def __init__(self, env):
        self.env_type = env
        self.env = gym.make(env)
        state, info = self.env.reset()
        self.policy_net = Net(len(state), self.env.action_space.n)
        self.optimizer = optim.AdamW(self.policy_net.parameters(), amsgrad=True)
        self.obs =len(state)
    
    def act(self, state):
        action = self.pick_action(state)
        state, reward, terminated, truncated, _ = self.env.step(action)
        return state, reward, terminated, truncated, _ 
        
        
    def pick_action(self, state):
        # Get probability list from policy net
        weights = self.policy_net(state)
        # Apply that list to the action list to get the appropriate action 
        action = torch.multinomial(weights, 1)#.todevice
        return action.item()
    
    
    def evaluation(self, values):
        # Need to update the actors policy with the critics evaluation 
        # use softmaxing...
        pass
    
    
    def change_render(self, render):
        if render:
            self.env = gym.make(self.env_type, render="human")
        else: 
            self.env = gym.make(self.env_type)

In [25]:
# Critic thread
class Critic:
    def __init__(self, obs, action):
        
        self.policy_net = Net(obs, action)
        self.optimizer = optim.AdamW(self.policy_net.parameters(),amsgrad=True )
    
    
    def evaluate(self, state, next_state, reward):
        # Need to generate an evaluation to update policy 
        # Calculate the Q value
        # Calculate the value function
        # Soft-maxing ???
        torch.gradient()
        pass

In [ ]:
# AC2 Agent for Cart Pole
environment = 'CartPole-v1'
agent = AC2(environment)

episodes = 10
gamma = 1.003

agent.actor.change_render(True)

# Main training session
total_rewards = agent.train(episodes, gamma)
print("Best reward: ", max(total_rewards))
agent.save("drpreisl_part1_assignment3.pickle")
reward_print(total_rewards, episodes, "grid world")

# Greedy run 
agent.actor.change_render(True)
total_greedy_rewards = agent.train(11, gamma, greedy=True)
reward_print(total_greedy_rewards, 10, "greedy")

In [8]:
environment = gym.make('CartPole-v1', render_mode="human")

state, info = environment.reset()

done = False
for i in range(100):
    if not done:
        state, reward, terminated, truncated, _  = environment.step(environment.action_space.sample())
        print(environment.action_space.sample())
        done = terminated or truncated
        # print(reward)
    else:
        break
environment.close()


TypeError: object of type 'Box' has no len()